# 7 합성곱 신경망(CNN)

## 7-1 전체 구조

## 7-2 합성곱 계층

### 7-2-1 완전연결 계층의 문제점

### 7-2-2 합성곱 연산

### 7-2-3 패딩

### 7-2-4 스트라이드

### 7-2-5 3차원 데이터의 합성곱 연산

### 7-2-6 블록으로 생각하기

### 7-2-7 배치처리

## 7-3 풀링 계층

### 7-3-1 풀링 계층의 특징

## 7-4 합성곱/ 풀링 계층 구현하기

### 7-4-1 4차원 배열

In [1]:
import numpy as np

In [2]:
x = np.random.rand(10 ,1 , 28, 28)

In [4]:
x.shape

(10, 1, 28, 28)

In [5]:
x[0,0]

array([[0.20318062, 0.90830099, 0.61623424, 0.45235407, 0.69053795,
        0.32703147, 0.26255872, 0.8994811 , 0.29549412, 0.70941728,
        0.60763675, 0.24714336, 0.97070881, 0.63000522, 0.16064803,
        0.3486442 , 0.60674609, 0.59485641, 0.85732157, 0.14261029,
        0.41943372, 0.81902423, 0.91319569, 0.07326052, 0.82932484,
        0.77103577, 0.83902237, 0.46026164],
       [0.25438811, 0.52824099, 0.11344433, 0.03409597, 0.42861967,
        0.36595667, 0.69649825, 0.02947823, 0.27153377, 0.66603061,
        0.46951514, 0.97466967, 0.03349602, 0.22057858, 0.93523733,
        0.89925888, 0.23917287, 0.6895084 , 0.69978187, 0.01751361,
        0.77683608, 0.8481643 , 0.88058918, 0.46323217, 0.49482589,
        0.76156395, 0.87282477, 0.69573747],
       [0.68379382, 0.91665018, 0.36974727, 0.88051079, 0.62240262,
        0.7239483 , 0.94338224, 0.66272508, 0.51744518, 0.78318027,
        0.07359149, 0.41375213, 0.41377974, 0.16587207, 0.57074746,
        0.02439068, 0.8478

### 7-4-2 im2col로 데이터 계산하기

### 7-4-3 합성곱 계층 구현하기

In [6]:
%cd /content/drive/MyDrive/AI/밑시딥 독학/모듈

/content/drive/MyDrive/AI/밑시딥 독학/모듈


In [7]:
from util import im2col

In [8]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5, 1, stride = 1, pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride = 1 , pad = 0)
print(col2.shape)

(21, 15)
(90, 75)


In [ ]:
class Convolution:
    def __init__(self, W, b , stride = 1 , pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N,C,H,W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x , FH , FW , self.stride, self.pad)
        col_W = self.W.reshape(FN , -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N , out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

        

### 7-4-4 풀링 계층 구현하기

In [10]:
class Pooling:
    def __init__ (self , pool_h , pool_w , stride = 1 , pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        #전개 (1)
        col = im2col(x, self.pool_h , self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        #최댓값 (2)
        out = np.max(col, axis=1)

        #성형 (3)
        out = out.reshape(N, out_h, out_w , C).transpose(0,3,1,2)

        return out

## 7-5 CNN 구현하기

In [15]:
from functions import *

In [20]:
class SimpleConvNet:
    def __init__(self, input_dim = (1,28,28),
                 conv_param={'filter_num':30, 'filter_size':5,
                             'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
                            filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size /2) * (conv_output_size / 2))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random,randn(filter_num , input_dip[0] , filter_size, filter_size)
        self. params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        #CNN을 구성하는 계층
        self.layers = OderedDict()
        self.layers['Conv1'] = Concolution(self.params['W1'], self.params['b1'], conv_param['stride'] , conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine1'] = Affine(self.params['W2'] , self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'] , self.params['b3'])

        self.last_layer = SoftmaxWithLoss

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forard(x)
        return x

    def loss(self, x ,t):
        y = self.predict(x)
        return self.last_layer.forward(y,t)

    def gradient(self, x, t):
        #순전파
        self.loss(x,t)

        #역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backard(dout)

        #결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads

## 7-6 CNN 시각화하기